In [2]:
from torchvision.models import ResNet50_Weights

import src.utils as utils
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [3]:
df = utils.build_dataset('data/plantvillage/plantvillage dataset')
df

,Format,Species,Healthy,Disease,Folder,FileName
0,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,8f558908-aa1b-4a86-855a-5094c2392e5a___RS_HL 1...
1,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,b8e9ed27-8e37-4214-9206-f8c0ef21cf4d___RS_HL 4...
2,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,abdd34a0-ab02-41e0-95a3-a014ab863ec2___RS_HL 1...
3,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,d1aee44a-b6bb-45b9-b7b6-5d553add8fd1___RS_HL 2...
4,color,Strawberry,True,None,data/plantvillage/plantvillage dataset/color/S...,3d28c3ea-8419-4e09-addd-211e3828e39f___RS_HL 1...
...,...,...,...,...,...,...
162911,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,f6579a78-e6eb-4a65-82f7-7be30f100a07___RS_HL 5...
162912,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,356eb227-3e6d-4164-b84d-31f590293644___RS_HL 4...
162913,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,5d3def53-fdb2-4106-ad31-c020e75bccea___RS_HL 7...
162914,segmented,Soybean,True,None,data/plantvillage/plantvillage dataset/segment...,63d474df-5512-4ecc-9cd3-c0649c260668___RS_HL 7...


In [4]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.df.iloc[idx]['Folder'], self.df.iloc[idx]['FileName'])
        image = Image.open(img_path).convert('RGB')
        label = 1 if self.df.iloc[idx]['Healthy'] else 0  # binary classification

        if self.transform:
            image = self.transform(image)
        return image, label


In [5]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [6]:
df = df[df['Healthy'].notna()]  # eliminamos filas sin etiqueta

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Healthy'], random_state=42)

train_dataset = PlantDataset(train_df, transform=train_transform)
val_dataset = PlantDataset(val_df, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [7]:
device = "cpu"
if torch.cuda.is_available():
  device = "cuda:0"

device

'cpu'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(weights=ResNet50_Weights.DEFAULT)

# Reemplazamos la última capa para clasificación binaria
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

model = model.to(device)

# Congelamos todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Activamos solo los parámetros de la última capa (la nueva capa fc)
for param in model.fc.parameters():
    param.requires_grad = True

for name, param in model.named_parameters():
    print(f"{name}: requires_grad = {param.requires_grad}")


In [ ]:
criterion = nn.BCEWithLogitsLoss()  # para clasificación binaria

optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)

for epoch in range(5):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")
